# withparentのコードリーディング

In [94]:
class treedict(dict):
    def __init__(self, iterable=(), parent=None, **kwargs):
        super(treedict, self).__init__(iterable, **kwargs)

In [95]:
t = treedict()

In [96]:
t

{}

In [97]:
t.__setitem__('a', 1)
t # => {'a': 1}

{'a': 1}

In [98]:
t.update({'b': 2})
t # => {'a': 1, 'b': 2}

{'a': 1, 'b': 2}

# wrapped

In [87]:
def withparent(meth):
    """Helper wrapper that passes calls to parent's instance"""
    def wrapped(self, *args, **kwargs):
        print("meth: ", meth) # <= debug
        print("meth.name: ", meth.__name__) # <= debug
        res = meth(self, *args, **kwargs)
        print("res: ", res) # <= debug
        if getattr(self, 'parent', None) is not None:
            print("parent: ", self.parent) # <= debug
            getattr(self.parent, meth.__name__)(*args, **kwargs)
        return res
    # Unfortunately functools wrapper fails
    # when decorating built-in methods so we
    # need to fix that improper behaviour
    print("wrapped.name before: ", wrapped.__name__) # <= debug
    wrapped.__name__ = meth.__name__
    print("wrapped.name after: ", wrapped.__name__) # <= debug
    return wrapped

In [99]:
class treedict(dict):
    
    def __init__(self, iterable=(), parent=None, **kwargs):
        super(treedict, self).__init__(iterable, **kwargs)
        assert isinstance(parent, dict) or parent is None
        self.parent = parent
        if self.parent is not None:
            self.parent.update(self)
        
    foo = withparent(dict.__setitem__)

wrapped.name before:  wrapped
wrapped.name after:  __setitem__


parentが存在しない場合

In [100]:
t = treedict()

In [101]:
t.foo('a', 1)
t # => {'a': 1}

meth:  <slot wrapper '__setitem__' of 'dict' objects>
meth.name:  __setitem__
res:  None


{'a': 1}

parentが存在する場合

In [102]:
t = treedict(parent={'x': 9})
t

{}

In [103]:
t.parent

{'x': 9}

In [104]:
t.foo('a', 1)
t # => {'a': 1}

meth:  <slot wrapper '__setitem__' of 'dict' objects>
meth.name:  __setitem__
res:  None
parent:  {'x': 9}


{'a': 1}

In [105]:
t.parent

{'a': 1, 'x': 9}